## Initiate Text Sample

In [67]:
# file = '../text_data/input.txt'
# file = open(file , 'r')
# text = file.read()

# Performing TD-IDF

## Importing Modules

In [68]:
import nltk
import codecs
import os
import re
import math
import operator

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk import PorterStemmer
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances_argmin_min
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn_extra.cluster import KMedoids

from gensim.models import Word2Vec
from scipy.spatial import distance
from string import punctuation

### Downloading libraries

In [69]:

nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hackintosh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hackintosh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hackintosh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hackintosh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [70]:
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()
def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
       lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words
def stem_words(words):
    stemmed_words = []
    for word in words:
       stemmed_words.append(PorterStemmer().stem(word))
    return stemmed_words
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text
def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
       if word not in words_unique:
           words_unique.append(word)
    for word in words_unique:
       dict_freq[word] = words.count(word)
    return dict_freq
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
             pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb
def tf_score(word,sentence):
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf
def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    # print('WORDS', word)
    for sentence in sentences:
        sentence = remove_special_characters(str(sentence))
        sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.split()
        sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
        sentence = [word.lower() for word in sentence]
        sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf
def tf_idf_score(tf,idf):
    return tf*idf
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf
def sentence_importance(sentence,dict_freq,sentences):
     sentence_score = 0
     sentence = remove_special_characters(str(sentence)) 
     sentence = re.sub(r'\d+', '', sentence)
     pos_tagged_sentence = [] 
     no_of_sentences = len(sentences)
     pos_tagged_sentence = pos_tagging(sentence)
     for word in pos_tagged_sentence:
          if word.lower() not in Stopwords and word not in Stopwords and len(word)>1: 
                word = word.lower()
                word = wordlemmatizer.lemmatize(word)
                sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
     return sentence_score
# file = '../text_data/input.txt'
# file = open(file , 'r')
# text = file.read()
def buildTfIdfSummary(text, retention_rate):
    summary = []
    sentence_no = []

    tokenized_sentence = sent_tokenize(text.rstrip('\n'))
    text = remove_special_characters(str(text))
    text = re.sub(r'\d+', '', text)
    tokenized_words_with_stopwords = word_tokenize(text)
    tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
    tokenized_words = [word for word in tokenized_words if len(word) > 1]
    tokenized_words = [word.lower() for word in tokenized_words]
    tokenized_words = lemmatize_words(tokenized_words)


    sentence_with_importance = getSentenceImportance(tokenized_sentence, tokenized_words)

    sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)

    addSentenceNumber(tokenized_sentence, sentence_with_importance, sentence_no, retention_rate)
    
    sentence_no.sort()

    summary = assembleWords(tokenized_sentence, sentence_no)
    return summary


def getSentenceImportance(tokenized_sentence, tokenized_words):
    sentence_with_importance = {}
    c = 1
    word_freq = freq(tokenized_words)
    for sent in tokenized_sentence:
        sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)
        sentence_with_importance[c] = sentenceimp
        c += 1
    return sentence_with_importance


def addSentenceNumber(tokenized_sentence, sentence_with_importance, sentence_no, retention_rate):
    cnt = 0
    no_of_sentences = int((retention_rate * len(tokenized_sentence))/100)
    for word_prob in sentence_with_importance:
        if cnt < no_of_sentences:
            sentence_no.append(word_prob[0])
            cnt = cnt+1
        else:
            break

def assembleWords(tokenized_sentence, sentence_no):
    cnt = 1
    summary = []
    for sentence in tokenized_sentence:
        if cnt in sentence_no:
            summary.append(sentence)
        cnt = cnt+1
    
    summary = " ".join(summary)
    return summary


## Word Vectorizing

In [71]:
def defineCorpus(sentence):
    corpus = []
    for i in range(len(sentence)):
        sen = re.sub('[^a-zA-Z]', " ", sentence[i])
        sen = sen.lower()
        sen = sen.split()
        sen = ' '.join([i for i in sen if i not in stopwords.words('english')])
        corpus.append(sen)
    return corpus


def vectorize(corpus):

    # creating word vectors
    n = 300
    all_words = [i.split() for i in corpus]
    model = Word2Vec(all_words, min_count=1, size=n)

    # creating sentence vectors
    sen_vector = []
    for i in corpus:
        plus = 0
        for j in i.split():
            plus += model.wv[j]
        plus = plus/len(plus)
        sen_vector.append(plus)
    return sen_vector

## K-Means

In [72]:
def performKMeans(text, cluster):
    sentence = sent_tokenize(text)
    corpus = defineCorpus(sentence)
    sentenceVector = vectorize(corpus)
    result = kMeansClustering(sentenceVector, cluster)
    summary = []
    for i in sorted(result):
        summary.append(sentence[i])
    summary = " ".join(summary)
    return summary

# performing k-means


def kMeansClustering(sen_vector, cluster):
    n_clusters = cluster
    kmeans = KMeans(n_clusters, init='k-means++', random_state=43)
    y_kmeans = kmeans.fit_predict(sen_vector)

    # finding and printing the nearest sentence vector from cluster centroid
    my_list = []
    for i in range(n_clusters):
        my_dict = {}

        for j in range(len(y_kmeans)):
            if y_kmeans[j] == i:
                my_dict[j] = distance.euclidean(
                    kmeans.cluster_centers_[i], sen_vector[j])
        my_list.append(min(my_dict, key=my_dict.get))

    return my_list

## K-Medoids

In [73]:
def performKMedoids(text, cluster):
    sentence = sent_tokenize(text)
    corpus = defineCorpus(sentence)
    sentenceVector = vectorize(corpus)
    result = kMedoidsClustering(sentenceVector, cluster)
    summary = []
    for i in sorted(result):
        summary.append(sentence[i])
    summary = " ".join(summary)
    return summary


def kMedoidsClustering(sen_vector, cluster):
    n_clusters = cluster
    kMedoids = KMedoids(n_clusters, random_state=43, method="pam")
    y_kmedoids = kMedoids.fit_predict(sen_vector)

    # finding and printing the nearest sentence vector from cluster centroid
    my_list = []
    for i in range(n_clusters):
        my_dict = {}

        for j in range(len(y_kmedoids)):
            if y_kmedoids[j] == i:
                my_dict[j] = distance.euclidean(
                    kMedoids.labels_[i], sen_vector[j])
        my_list.append(min(my_dict, key=my_dict.get))

    return my_list

## Cosine Similarities

In [74]:
def find_similarities(text):
    # tokenize sentences
    sentences = sent_tokenize(text)
    #set stop words
    stops = list(set(stopwords.words('english'))) + list(punctuation)
    
    #vectorize sentences and remove stop words
    vectorizer = TfidfVectorizer(stop_words = stops)
    #transform using TFIDF vectorizer
    trsfm=vectorizer.fit_transform(sentences)
    
    #creat df for input article
    text_df = pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=sentences)
    
    #declare how many sentences to use in summary
    num_sentences = text_df.shape[0]
    num_summary_sentences = int(np.ceil(num_sentences**.5))
        
    #find cosine similarity for all sentence pairs
    similarities = cosine_similarity(trsfm, trsfm)
    
    #create list to hold avg cosine similarities for each sentence
    avgs = []
    for i in similarities:
        avgs.append(i.mean())
     
    #find index values of the sentences to be used for summary
    top_idx = np.argsort(avgs)[-num_summary_sentences:]
    
    return top_idx

def performCosineSimilarity(text):
    #find sentences to extract for summary
    sents_for_sum = find_similarities(text)
    #sort the sentences
    sort = sorted(sents_for_sum)
    #display which sentences have been selected
    # print(sort)
    
    sent_list = sent_tokenize(text)
    #print number of sentences in full article
    # print(len(sent_list))
    
    
    #extract the selected sentences from the original text
    sents = []
    for i in sort:
        sents.append(sent_list[i].replace('\n', ''))
    
    #join sentences together for final output
    summary = ' '.join(sents)
    return summary

In [93]:
file = '../text_data/input3.txt'
file = open(file , 'r')
text = file.read()

In [98]:
tfidfSummary = buildTfIdfSummary(text, 60)
print('\nOriginal Text: \n\n', text)
print('\nTF-IDF Applied: \n\n', tfidfSummary)


Original Text: 

 Sentence representation at the semantic level is a challenging task for natural language processing and Artificial Intelligence.
Despite the advances in word embeddings (i.e. word vector representations), capturing sentence meaning is an open question due to complexities of semantic interactions among words. In this paper, we present an embedding method, which is aimed at learning unsupervised sentence representations from unlabeled text. We propose an unsupervised method that models a sentence as a weighted series of word embeddings. The weights of the series are fitted by using Shannon’s Mutual Information (MI) among words, sentences and the corpus. In fact, the Term Frequency?Inverse Document Frequency transform (TF?IDF) is a reliable estimate of such MI. Our method offers advantages over existing ones: identifiable modules, short-term training, online inference of (unseen) sentence representations, as well as independence from domain, external knowledge and lingu

In [78]:
kmeansSummary = performKMeans(tfidfSummary, 7)
cosineSimilaritySummary = performCosineSimilarity(kmeansSummary)

print('\nK-Means Applied: \n\n', kmeansSummary)
print('\nCosine Similarities Applied: \n\n', cosineSimilaritySummary)

ValueError: n_samples=4 should be >= n_clusters=7.

In [80]:

kmedoidsSummary = performKMedoids(tfidfSummary, 7)
cosineSimilaritySummary = performCosineSimilarity(kmedoidsSummary)

print('\nK-Medoids Result: \n\n', kmedoidsSummary)
print('\nCosine Similarities Result: \n\n', cosineSimilaritySummary)


ValueError: The number of medoids (7) must be less than the number of samples 4.